In [3]:
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop, Adam
import os

In [9]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量，定義訓練週期

# 讀取資料並檢視
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 對 label 進行 one-hot encoding (y_trian 原本是純數字)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## 首先我們使用一般的 DNN (MLP) 來訓練
由於 DNN 只能輸入一維的資料，我們要先將影像進行攤平，若 (50000, 32, 32, 3) 的影像，攤平後會變成 (50000, 32 x32 x 3) = (50000, 3072)

In [3]:
# 將資料攤平成一維資料
x_train = x_train.reshape(50000, 3072) 
x_test = x_test.reshape(10000, 3072)

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

50000 train samples
10000 test samples


In [4]:
model = Sequential() # 建立簡單的線性執行的模型
model.add(Dense(512, activation='relu', input_shape=(3072,))) # 全連接層: 512個output變數
model.add(Dropout(0.2)) # Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.2。
model.add(Dense(512, activation='relu')) # 全連接層: 512個output變數
model.add(Dropout(0.2))  # Dropout層隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.2。
model.add(Dense(num_classes, activation='softmax')) # 全連接層: 10個output變數，使用 softmax activation function，將結果分類
model.summary()

# 編譯: 選擇損失函數、優化方法及成效衡量方式
# 確立目標及求解方法：以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# 訓練：以compile函數進行訓練，指定訓練的樣本資料(x, y)，並撥一部分資料作驗證，還有要訓練幾個週期、訓練資料的抽樣方式。
# 訓練過程會存在 history 變數中
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs, 
                    verbose=1,
                    validation_data=(x_test, y_test))

# 顯示損失函數、訓練成果(分數)
# verbose：日誌顯示，0為不在標準輸出流輸出日誌信息，1為輸出進度條記錄，2為每個epoch輸出一行記錄。
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Instructions for updating:
Colocations handled automatically by placer.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5130      
Total params: 1,841,162
Trainable params: 1,841,162
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast

## 接下來我們使用 CNN 來訓練神經網路
CNN 的原理非常適合處理影像類的資料，就讓我們來看看，同樣的訓練條件，CNN 是否顯著優於 DNN 呢?

In [10]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# 將資料變為 float32 並標準化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# 轉換色彩 0~255 資料為 0~1
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
# y 值轉成 one-hot encoding，轉成10類
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [11]:
# 建立簡單的線性執行的模型
model = Sequential()
# 建立卷積層，filter=32,即 output space 的深度, Kernal Size: 3x3, padding='same'不足的空間用 0填充一列
model.add(Conv2D(32, (3, 3), padding='same',input_shape=x_train.shape[1:]))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立卷積層，filter=32,,即 output size, Kernal Size: 3x3
model.add(Conv2D(32, (3, 3)))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# 隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3, padding='same'不足的空間用 0填充一列
model.add(Conv2D(64, (3, 3), padding='same'))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立卷積層，filter=64,即 output size, Kernal Size: 3x3
model.add(Conv2D(64, (3, 3)))
# activation function 採用 relu
model.add(Activation('relu'))
# 建立池化層，池化大小=2x2，取最大值
model.add(MaxPooling2D(pool_size=(2, 2)))
# 隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.25
model.add(Dropout(0.25))

# 把多維的輸入一維化，常用在從卷積層到全連接層的過渡。
model.add(Flatten())
# 全連接層: 512個output
model.add(Dense(512))
# activation function 採用 relu
model.add(Activation('relu'))
# 隨機斷開輸入神經元，用於防止過度擬合，斷開比例:0.5
model.add(Dropout(0.5))
# 使用 softmax activation function，並將結果分類
model.add(Dense(num_classes))
model.add(Activation('softmax'))
# 輸出模型概況
model.summary()

# 編譯: 選擇損失函數、優化方法及成效衡量方式
# 確立目標及求解方法：以compile函數定義損失函數(loss)、優化函數(optimizer)及成效衡量指標(mertrics)。
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

# 訓練：以compile函數進行訓練，指定訓練的樣本資料(x, y)，並撥一部分資料作驗證，還有要訓練幾個週期、訓練資料的抽樣方式。
# 訓練過程會存在 history 變數中
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

# 顯示損失函數、訓練成果(分數)
# verbose：日誌顯示，0為不在標準輸出流輸出日誌信息，1為輸出進度條記錄，2為每個epoch輸出一行記錄。
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 15, 15, 64)       

In [12]:
# 模型結構存檔
from keras.models import model_from_json
json_string = model.to_json()
with open("cnn.config", "w") as text_file:
    text_file.write(json_string)

# 模型訓練結果存檔
model.save_weights("cnn.weight")

## 同樣運算 10 個 epochs，但 CNN 在 test data 的準確率顯著優於 DNN!

## 作業
1. 請試著調整各個超參數，並說明那些超參數對於結果有明顯的影響?
2. CNN 與 DNN 哪個模型的參數數量比較多? 造成參數的數量不同的原因在哪?

# Ans

1. batch size:會影響收斂、drop_out:會模型結果有一些影響。

2. CNN模型的參數數量:1250858、DNN模型的參數數量:1841162，兩者相比DNN模型的參數數量比較多，其可能原因在於CNN在做捲積時，已經先減少參數數量，縮小了特徵圖大小。

# 參考解答
1. 可以調整的有: Conv2D 的層數、filters 數量、stride 大小、學習率的大小、優化器的選擇、Dropout 的比率等等。其中 Conv2D 的層數與優化器的選擇對結果會有較明顯的影響。
2. DNN 的參數量較多，可以看出 CNN 在做卷機時，透過較小的參數量就可以逐步縮小影像大小，讓最後再接上 Dense 層時不要有過多的參數。